# Démonstration : TOTP (Time-Based One-Time Password)

Implémentation du standard RFC 6238 utilisé par Google Authenticator, Microsoft Authenticator, etc.

**Principe** : Générer un code à 6 chiffres basé sur l'heure actuelle et une clé secrète partagée.

In [ ]:
import hmac
import base64
import struct
import time
import secrets
from urllib.parse import quote

# Installer pyotp pour comparaison
try:
    import pyotp
except ImportError:
    !pip install pyotp
    import pyotp

## 1. Implémentation TOTP from scratch

In [ ]:
def generate_totp(secret, time_step=30, digits=6):
    """
    Génère un code TOTP selon RFC 6238.
    
    Args:
        secret: Clé secrète (bytes ou base32 string)
        time_step: Durée de validité en secondes (défaut 30s)
        digits: Nombre de chiffres du code (défaut 6)
    """
    # Si secret est une string base32, la décoder
    if isinstance(secret, str):
        secret = base64.b32decode(secret.upper())
    
    # Calculer le compteur basé sur le temps
    counter = int(time.time() / time_step)
    
    # Convertir le compteur en bytes (8 bytes, big-endian)
    counter_bytes = struct.pack('>Q', counter)
    
    # Calculer HMAC-SHA1
    hmac_hash = hmac.new(secret, counter_bytes, 'sha1').digest()
    
    # Dynamic Truncation (RFC 6238 section 5.3)
    offset = hmac_hash[-1] & 0x0F
    truncated = hmac_hash[offset:offset+4]
    
    # Convertir en entier et prendre les N derniers chiffres
    code = struct.unpack('>I', truncated)[0] & 0x7FFFFFFF
    code = code % (10 ** digits)
    
    return f"{code:0{digits}d}"

def verify_totp(secret, code, time_step=30, window=1):
    """
    Vérifie un code TOTP avec une fenêtre de tolérance.
    
    Args:
        window: Nombre de time_steps avant/après à accepter (défaut 1 = +/- 30s)
    """
    for i in range(-window, window + 1):
        # Ajuster le temps pour la fenêtre
        adjusted_time = time.time() + (i * time_step)
        counter = int(adjusted_time / time_step)
        
        # Générer le code pour ce time step
        if isinstance(secret, str):
            secret_bytes = base64.b32decode(secret.upper())
        else:
            secret_bytes = secret
            
        counter_bytes = struct.pack('>Q', counter)
        hmac_hash = hmac.new(secret_bytes, counter_bytes, 'sha1').digest()
        offset = hmac_hash[-1] & 0x0F
        truncated = hmac_hash[offset:offset+4]
        expected_code = struct.unpack('>I', truncated)[0] & 0x7FFFFFFF
        expected_code = f"{expected_code % 1000000:06d}"
        
        if code == expected_code:
            return True
    return False

# Test
secret = base64.b32encode(secrets.token_bytes(20)).decode()
print(f"Clé secrète (Base32) : {secret}")
print(f"\nCode TOTP actuel : {generate_totp(secret)}")
print(f"Durée de validité : 30 secondes")

# Afficher plusieurs codes consécutifs
print("\nCodes pour les prochaines périodes :")
for i in range(5):
    timestamp = int(time.time()) + (i * 30)
    counter = timestamp // 30
    secret_bytes = base64.b32decode(secret)
    counter_bytes = struct.pack('>Q', counter)
    hmac_hash = hmac.new(secret_bytes, counter_bytes, 'sha1').digest()
    offset = hmac_hash[-1] & 0x0F
    truncated = hmac_hash[offset:offset+4]
    code = struct.unpack('>I', truncated)[0] & 0x7FFFFFFF
    code = f"{code % 1000000:06d}"
    print(f"  T+{i*30:3d}s : {code}")

## 2. Génération de QR Code pour Google Authenticator

In [ ]:
try:
    import qrcode
    from PIL import Image
except ImportError:
    !pip install qrcode[pil]
    import qrcode
    from PIL import Image

def generate_qr_code(secret, username="user@example.com", issuer="MyApp"):
    """
    Génère un QR code compatible Google Authenticator.
    
    Format URI : otpauth://totp/Label?secret=SECRET&issuer=ISSUER
    """
    # Construire l'URI otpauth
    label = f"{issuer}:{username}"
    uri = f"otpauth://totp/{quote(label)}?secret={secret}&issuer={quote(issuer)}"
    
    # Générer QR code
    qr = qrcode.QRCode(version=1, box_size=10, border=4)
    qr.add_data(uri)
    qr.make(fit=True)
    
    img = qr.make_image(fill_color="black", back_color="white")
    return img, uri

# Générer et afficher le QR code
img, uri = generate_qr_code(secret, "alice@example.com", "DemoApp")
print(f"URI otpauth : {uri}")
print("\nScannez ce QR code avec Google Authenticator :")
display(img)

print("\n✅ Instructions :")
print("  1. Ouvrez Google Authenticator sur votre téléphone")
print("  2. Appuyez sur + puis 'Scanner un code QR'")
print("  3. Scannez le QR code ci-dessus")
print("  4. Le code à 6 chiffres s'affichera et se renouvellera toutes les 30s")

## 3. Comparaison avec pyotp (librairie standard)

In [ ]:
# Utiliser pyotp
totp = pyotp.TOTP(secret)

# Comparer notre implémentation avec pyotp
our_code = generate_totp(secret)
pyotp_code = totp.now()

print(f"Notre implémentation : {our_code}")
print(f"pyotp (librairie)    : {pyotp_code}")
print(f"\nIdentiques : {our_code == pyotp_code}")
print("\n✅ Notre implémentation est conforme au standard RFC 6238 !")

## 4. Test de vérification avec fenêtre de tolérance

In [ ]:
# Générer un code actuel
current_code = generate_totp(secret)
print(f"Code actuel : {current_code}")

# Vérifier immédiatement
is_valid = verify_totp(secret, current_code, window=1)
print(f"Vérification immédiate : {is_valid}")

# Simuler un code de la période précédente
old_timestamp = int(time.time()) - 30
old_counter = old_timestamp // 30
secret_bytes = base64.b32decode(secret)
counter_bytes = struct.pack('>Q', old_counter)
hmac_hash = hmac.new(secret_bytes, counter_bytes, 'sha1').digest()
offset = hmac_hash[-1] & 0x0F
truncated = hmac_hash[offset:offset+4]
old_code = f"{(struct.unpack('>I', truncated)[0] & 0x7FFFFFFF) % 1000000:06d}"

print(f"\nCode de la période précédente (-30s) : {old_code}")
is_valid_old = verify_totp(secret, old_code, window=1)
print(f"Accepté avec window=1 : {is_valid_old}")

# Code invalide
invalid_code = "000000"
is_valid_invalid = verify_totp(secret, invalid_code, window=1)
print(f"\nCode invalide accepté : {is_valid_invalid}")

print("\n💡 La fenêtre de tolérance (window) compense les décalages d'horloge")

## 5. Démonstration temps réel

In [ ]:
import sys
from IPython.display import clear_output

print("Affichage du code TOTP en temps réel (Ctrl+C pour arrêter):\n")

try:
    for _ in range(10):  # 10 itérations (5 minutes)
        current_code = generate_totp(secret)
        
        # Calculer secondes restantes
        seconds_left = 30 - (int(time.time()) % 30)
        
        # Afficher
        clear_output(wait=True)
        print(f"Code TOTP : {current_code}")
        print(f"Expire dans : {seconds_left} secondes")
        print("\n" + "█" * seconds_left + "░" * (30 - seconds_left))
        
        time.sleep(1)
except KeyboardInterrupt:
    print("\nArrêté")

## Conclusion

**Points clés** :
- TOTP utilise HMAC-SHA1 sur un compteur temporel
- Les codes changent toutes les 30 secondes
- Une fenêtre de tolérance compense les décalages d'horloge
- Compatible avec Google Authenticator, Microsoft Authenticator, etc.
- Beaucoup plus sécurisé que SMS pour 2FA